<a href="https://colab.research.google.com/github/pabloperfer/CUDAexercises/blob/main/accelerateneural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
from numba import cuda, vectorize

n = 100000000

greyscales = np.floor(np.random.uniform(0, 255, n).astype(np.float32))
weights = np.random.normal(.5, .1, n).astype(np.float32)

In [15]:
from math import exp

In [16]:
from numba import vectorize
@vectorize(['float32(float32)'], target='cuda')
def normalize(grayscales):
    return grayscales / 255

@vectorize(['float32(float32,float32)'], target='cuda')
def weigh(values, weights):
    return values * weights
        
@vectorize(['float32(float32)'], target='cuda')    
def activate(values):
    return ( exp(values) - exp(-values) ) / ( exp(values) + exp(-values) )

In [17]:
def create_hidden_layer(n, greyscales, weights, exp, normalize, weigh, activate):
    d_greyscales = cuda.to_device(greyscales)
    d_weights = cuda.to_device(weights)
    
    normalized = normalize(d_greyscales)
    weighted = weigh(normalized, d_weights)
    activated = activate(weighted)
    
    out_host = activated.copy_to_host()
    return activated

In [18]:
arguments = {"n":n,
            "greyscales": greyscales,
            "weights": weights,
            "exp": exp,
            "normalize": normalize,
            "weigh": weigh,
            "activate": activate}
a = create_hidden_layer(**arguments)
print(a)